In [1]:
import keras as K
import numpy as np
import os
from keras.utils import to_categorical
import _pickle as pickle
from pathlib import Path
from models import *
from datautils import *
import tensorflow as tf

/home/deep/pythonenv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

%reload_ext autoreload
%autoreload 2

In [3]:
word_embed_weight=np.load('./cache/word_embed.npy')
train_text = np.load('./cache/train_text.npy')
train_label = np.load('./cache/train_label.npy')
val_y = np.load('./cache/val_label.npy')
val_X = np.load('./cache/val_X.npy')

In [4]:
train_label.shape, val_y.shape

((130687,), (18669,))

# Divide Pos and Neg

In [5]:
train_PN_idx = (train_label != 2)
train_PN_label = train_label[train_PN_idx]
val_PN_idx = (val_y != 2)
val_PN_label = val_y[val_PN_idx]
train_PN_label.shape, val_PN_label.shape

((111741,), (15975,))

In [6]:
train_text = train_text[train_PN_idx]
train_label = train_PN_label < 2

In [7]:
val_X = val_X[val_PN_idx]
val_y = to_categorical(val_PN_label < 2)

In [15]:
train_text.shape, val_X.shape, train_label.shape, val_y.shape

((111741,), (15975,), (111741,), (15975, 2))

In [9]:
print('Pos: ', sum(train_label==0))
print('Neg: ', sum(train_label==1))

Pos:  81222
Neg:  30519


In [10]:
class_nb = val_y.shape[1]
class_nb

2

In [14]:
val_text = np.load('./cache/val_text.npy')
val_X = val_text[val_PN_idx]

In [16]:
np.save('./cache/PN_train_text.npy',train_text)
np.save('./cache/PN_train_label.npy', train_label)
np.save('./cache/PN_val_X.npy', val_X)
np.save('./cache/PN_val_y.npy', val_y)

# Train

## TextCNN

In [11]:
batch_size=128
model_name='textcnn'
model=get_textcnn_v2(sentence_length, word_embed_weight, class_nb)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 150, 256)     11369216    input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 150, 256)     65792       embedding[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 150, 256)     1024        time_distributed_1[0][0]         
__________________________________________________________________________________________________
activation

In [12]:
train_batch_generator = word_text_train_batch_generator

In [13]:
for i in range(3):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
    pred = np.squeeze(model.predict(val_X))
    pre,rec,f1 = score(pred, val_y)
    print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 31s 35ms/step - loss: 0.4125 - acc: 0.8115 - val_loss: 0.3541 - val_acc: 0.8416


NameError: name 'val_seq' is not defined

In [14]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 29s 34ms/step - loss: 0.3564 - acc: 0.8415 - val_loss: 0.3583 - val_acc: 0.8347
0.6750048856752003 0.7793321299638989 0.7234265368101372


In [15]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 30s 34ms/step - loss: 0.3414 - acc: 0.8484 - val_loss: 0.3315 - val_acc: 0.8553
0.7560386473429952 0.7062274368231047 0.730284647690154


In [17]:
K.set_value(model.optimizer.lr, 0.0001)
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 31s 35ms/step - loss: 0.2698 - acc: 0.8863 - val_loss: 0.3143 - val_acc: 0.8645
0.7538633818589026 0.7594765342960289 0.7566595481623019


In [16]:
model.layers[1].trainable = True
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 31s 36ms/step - loss: 0.3277 - acc: 0.8560 - val_loss: 0.3182 - val_acc: 0.8601
0.7606168446026097 0.7233754512635379 0.7415288539377818


In [18]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 29s 34ms/step - loss: 0.2463 - acc: 0.8968 - val_loss: 0.3227 - val_acc: 0.8617
0.7357809847198642 0.7822653429602888 0.7583114610673666


## RNN

In [36]:
model.layers[1].trainable = True
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
872/872 [==============================] - 82s 94ms/step - loss: 0.2290 - acc: 0.9032 - val_loss: 0.3830 - val_acc: 0.8400


In [13]:
batch_size=128
model_name='textrnn'
model=get_rnn(sentence_length, word_embed_weight, class_nb)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 256)          11369216  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 150, 256)          65792     
_________________________________________________________________
batch_normalization_3 (Batch (None, 150, 256)          1024      
_________________________________________________________________
activation_3 (Activation)    (None, 150, 256)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                61632     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
__________

In [14]:
train_batch_generator = word_text_train_batch_generator

In [15]:
for i in range(3):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
    pred = np.squeeze(model.predict(val_X))
    pre,rec,f1 = score(pred, val_y)
    print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 97s 111ms/step - loss: 0.5661 - acc: 0.7358 - val_loss: 0.3954 - val_acc: 0.8219
0.6758253933082207 0.6881768953068592 0.6819452207937394
Epoch 1/1
872/872 [==============================] - 96s 110ms/step - loss: 0.3630 - acc: 0.8388 - val_loss: 0.3426 - val_acc: 0.8506
0.7541635595326871 0.6845667870036101 0.7176818450620935
Epoch 1/1
872/872 [==============================] - 96s 111ms/step - loss: 0.3336 - acc: 0.8527 - val_loss: 0.3282 - val_acc: 0.8575
0.7909802862543883 0.6608754512635379 0.7200983405039951


In [16]:
K.set_value(model.optimizer.lr, 0.0001)
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 97s 111ms/step - loss: 0.3083 - acc: 0.8664 - val_loss: 0.3245 - val_acc: 0.8575
0.7286229577763632 0.7748194945848376 0.7510114816839805


In [17]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 95s 109ms/step - loss: 0.3002 - acc: 0.8694 - val_loss: 0.3223 - val_acc: 0.8567
0.7261025532812829 0.7763989169675091 0.7504088976120379


In [18]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 96s 110ms/step - loss: 0.2958 - acc: 0.8714 - val_loss: 0.3197 - val_acc: 0.8592
0.7353892602976062 0.769404332129964 0.7520123497629287


## HAN

In [11]:
sen_seq_length = 10
word_seq_length = 15

In [12]:
val_text = np.load('./cache/val_text.npy')
val_X = word_han_preprocess(val_text)

In [13]:
val_X = val_X[val_PN_idx]

In [14]:
train_batch_generator = word_han_train_batch_generator
batch_size=128
model_name='han'
model=get_han(sen_seq_length, word_seq_length, word_embed_weight, class_nb)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 15)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 256)           11665167  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 256)           295680    
_________________________________________________________________
attention_2 (Attention)      (None, 256)               266       
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
__________

In [15]:
for i in range(2):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
    pred = np.squeeze(model.predict(val_X))
    pre,rec,f1 = score(pred, val_y)
    print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 83s 95ms/step - loss: 0.3784 - acc: 0.8309 - val_loss: 0.4026 - val_acc: 0.8293
0.8347074990184531 0.4796931407942238 0.6092563404499213
Epoch 1/1
872/872 [==============================] - 82s 93ms/step - loss: 0.3111 - acc: 0.8658 - val_loss: 0.3546 - val_acc: 0.8475
0.7430589381393083 0.6884025270758123 0.7146872803935347


In [16]:
for i in range(2):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
    pred = np.squeeze(model.predict(val_X))
    pre,rec,f1 = score(pred, val_y)
    print(pre,rec,f1)

Epoch 1/1
872/872 [==============================] - 81s 93ms/step - loss: 0.2470 - acc: 0.8962 - val_loss: 0.3699 - val_acc: 0.8441
0.719385449615906 0.7184115523465704 0.7188981711447279
Epoch 1/1
872/872 [==============================] - 82s 94ms/step - loss: 0.1827 - acc: 0.9256 - val_loss: 0.4817 - val_acc: 0.8344
0.7890650274991912 0.5503158844765343 0.6484115379502858


In [ ]:
K.set_value(model.optimizer.lr, 0.0001)
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))
pred = np.squeeze(model.predict(val_X))
pre,rec,f1 = score(pred, val_y)
print(pre,rec,f1)

In [ ]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
756/872 [=========================>....] - ETA: 10s - loss: 0.1949 - acc: 0.9184